# Description

This notebook contains tools for debugging an individual `CcxtBroker` execution.

The notebook is responsible for aspects of the execution experiment run that are related to the correctness of the execution, as opposed to the analysis of the result:
- Consistency of logging
- Correctness in the submission of orders
- Consistency between trades and submitted orders
- Order time profiling

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import logging

import core.config as cconfig
import helpers.hdbg as hdbg
import helpers.henv as henv
import helpers.hprint as hprint
import im_v2.common.universe as ivcu
import oms.broker.ccxt.ccxt_aggregation_functions as obccagfu
import oms.broker.ccxt.ccxt_execution_quality as obccexqu
import oms.broker.ccxt.ccxt_logger as obcccclo

In [ ]:
hdbg.init_logger(verbosity=logging.INFO)

_LOG = logging.getLogger(__name__)

_LOG.info("%s", henv.get_system_signature()[0])

hprint.config_notebook()

In [ ]:
config = cconfig.get_config_from_env()
if config:
    # Get config from env when running the notebook via the `run_notebook.py` script, e.g.
    # in the system reconciliation flow.
    _LOG.info("Using config from env vars")
else:
    system_log_dir = "/shared_data/ecs/test/system_reconciliation/C11a/prod/20240312_220000.20240312_225500/system_log_dir.manual/process_forecasts"
    config_dict = {"system_log_dir": system_log_dir}
    config = cconfig.Config.from_dict(config_dict)
print(config)

In [ ]:
log_dir = config["system_log_dir"]
ccxt_log_reader = obcccclo.CcxtLogger(log_dir)
#
data = ccxt_log_reader.load_all_data(
    convert_to_dataframe=True, abort_on_missing_data=False
)
ccxt_order_response_df = data["ccxt_order_responses"]
ccxt_trades_df = data["ccxt_trades"]
oms_child_order_df = data["oms_child_orders"]
oms_parent_order_df = data["oms_parent_orders"]
ccxt_fills = data["ccxt_fills"]

In [ ]:
# Print the Broker config.
if "broker_config" in data:
    print(hprint.to_pretty_str(data["broker_config"]))
else:
    _LOG.warning("broker_config file not present in %s", log_dir)

In [ ]:
# Print the used Config, if any.
experiment_config = obcccclo.load_config_for_execution_analysis(log_dir)
print(experiment_config)

In [ ]:
ccxt_order_response_df.head(3)

## Child order responses

In [ ]:
ccxt_order_response_df.info()

## Child orders

In [ ]:
oms_child_order_df.info()

In [ ]:
oms_child_order_df.head(3)

### Extract the OMS parent order ID

In [ ]:
oms_child_order_df["parent_order_id"] = oms_child_order_df["extra_params"].apply(
    lambda x: x["oms_parent_order_id"]
)
oms_child_order_df.head(3)

In [ ]:
# Check how many child orders are generated per parent_order_id.
oms_child_order_df["parent_order_id"].value_counts().value_counts()

### Check the error messages in non-submitted orders.

In [ ]:
# Select the OMS child orders with no CCXT ID and check their error messages.
not_submitted_oms_child_order_df = oms_child_order_df.loc[
    oms_child_order_df["ccxt_id"] == -1
]
print(
    f"Number of not submitted OMS child orders={not_submitted_oms_child_order_df.shape[0]} \
    out of total orders={oms_child_order_df.shape[0]}"
)

In [ ]:
# Extract all error messages from unsubmitted orders.
not_submitted_oms_child_order_df["error_msg"] = not_submitted_oms_child_order_df[
    "extra_params"
].apply(lambda x: x["error_msg"])

In [ ]:
# Display error messages grouped by symbol.
# Get the universe to map asset_id's.
universe = ivcu.get_vendor_universe("CCXT", "trade", as_full_symbol=True)
asset_id_to_symbol_mapping = ivcu.build_numerical_to_string_id_mapping(universe)
not_submitted_oms_child_order_df[
    "full_symbol"
] = not_submitted_oms_child_order_df["asset_id"].map(asset_id_to_symbol_mapping)
# Get value counts of error messages.
error_msg = not_submitted_oms_child_order_df.groupby("full_symbol")[
    "error_msg"
].value_counts()
error_msg

### Check the buy and sell orders with max notional

In [ ]:
oms_child_order_notionals = (
    oms_child_order_df["diff_num_shares"] * oms_child_order_df["limit_price"]
)
max_sell_notional = oms_child_order_notionals[oms_child_order_notionals < 0].min()
max_buy_notional = oms_child_order_notionals[oms_child_order_notionals > 0].max()
_LOG.info(
    "Max sell notional: %s\nMax buy notional: %s",
    max_sell_notional,
    max_buy_notional,
)

## Parent orders

In [ ]:
oms_parent_order_df.head(3)

In [ ]:
# Extract `ccxt_id` of child orders into a separate column.
oms_parent_order_df["child_order_ccxt_ids"] = oms_parent_order_df[
    "extra_params"
].apply(lambda x: x["ccxt_id"])
oms_parent_order_df.head(3)

In [ ]:
# Check how many child orders per parent order were successfully submitted.
# A value between maximum expected and 0 can mean that some child orders
# could not get through, while otherwise being well-formed.
# A value of 0 means that the parent order had an issue, e.g.
# having a notional that made all child orders below a minimum.
oms_parent_order_df["submitted_child_order_num"] = oms_parent_order_df[
    "child_order_ccxt_ids"
].str.len()
oms_parent_order_df["submitted_child_order_num"].value_counts()

## Trades

In [ ]:
ccxt_trades_df.info()

### Verify that all trades correspond to logged child orders

In [ ]:
# Verify that all trades have an associated order response.
# If the trade does not have an associated order, it means
# that a trade was executed that was not accounted for during the experiment.
all(ccxt_trades_df["order"].isin(ccxt_order_response_df["order"]))

In [ ]:
# Verify that all trades have an associated 'ccxt_id' in OMS child orders.
# This verifies that there is a consistency between a CCXT order, OMS order and
# a CCXT trade. If a trade has no associated OMS child order `ccxt_id`, it means
# that an unexpected trade was executed, for example, by a different actor on the same account.
all(ccxt_trades_df["order"].isin(oms_child_order_df["ccxt_id"]))

### Check orders that do not correspond to any trades

In [ ]:
# Existence of such orders is not necessarily a bug.
# It means that a given OMS child order was not filled.
child_orders_with_no_trades = ~oms_child_order_df["ccxt_id"].isin(
    ccxt_trades_df["order"]
)
child_orders_with_no_trades.sum()

### Check the correctness of trade amount

In [ ]:
# If the traded amount is larger than the order, it means that an extra trade
# is executed and the order is filled for a larger amount.
# This can mean an accounting error on the exchange side.
trade_amount_by_order = ccxt_trades_df.groupby("order").agg({"amount": "sum"})
accepted_child_order_df = oms_child_order_df.loc[
    oms_child_order_df["ccxt_id"] != -1
]
trade_amount_by_order["child_order_amount"] = accepted_child_order_df.set_index(
    "ccxt_id"
)["diff_num_shares"].abs()

In [ ]:
mask = (
    trade_amount_by_order["amount"] > trade_amount_by_order["child_order_amount"]
)
trade_amount_by_order.loc[mask]

In [ ]:
# Check the difference between the filled amount and the agg child order quantities.
trade_amount_by_order["diff"] = (
    trade_amount_by_order["child_order_amount"] - trade_amount_by_order["amount"]
)
trade_amount_by_order = trade_amount_by_order.rename(
    {"amount": "aggregated_trades_quantity"}, axis=1
)
trade_amount_by_order = trade_amount_by_order[
    ["child_order_amount", "aggregated_trades_quantity", "diff"]
]
trade_amount_by_order[trade_amount_by_order["diff"] > 0]

### Verify that CCXT IDs are equal in both child orders and responses

In [ ]:
# Verify that we are not logging any CCXT orders that are not a part
# of the experiment, i.e. have no OMS order associated with it.
# We expect all retrieved responses to be a subset of CCXT IDs
# connected to the OMS child orders.
submitted_oms_orders_ccxt_ids = set(
    oms_child_order_df.loc[oms_child_order_df["ccxt_id"] != -1][
        "ccxt_id"
    ].unique()
)
ccxt_response_ids = set(ccxt_order_response_df["order"].unique())
ccxt_response_ids.issubset(submitted_oms_orders_ccxt_ids)

# Group trades by order

In [ ]:
# Aggregate fills by order.
ccxt_trades_by_order = obccagfu.aggregate_fills_by_order(ccxt_trades_df)
ccxt_trades_by_order.head(3)

# Time profiling for child orders

In [ ]:
oms_child_order_df_unpacked = ccxt_log_reader.load_oms_child_order(
    unpack_extra_params=True, convert_to_dataframe=True
)

In [ ]:
# Add timestamp of when the order was closed.
order_lifespan_end = ccxt_fills.set_index("order")[
    "order_update_datetime"
].to_dict()
oms_child_order_df_unpacked["end_order_timestamp"] = oms_child_order_df_unpacked[
    "ccxt_id"
].apply(lambda x: order_lifespan_end.get(x))

In [ ]:
# Get the timestamps of events for each child order
events = obccexqu.get_oms_child_order_timestamps(oms_child_order_df_unpacked)
events = events.sort_values(events.first_valid_index(), axis=1)

In [ ]:
events.head(3)

In [ ]:
# Get the difference between event timestamps.
time_delays = obccexqu.get_time_delay_between_events(events)

In [ ]:
time_delays.boxplot(rot=45, ylabel="Time delay")

## Plot zero and non-zero waves separately

### Plot zero wave

In [ ]:
# Since wave 0 begins execution later due to portfolio computation,
# we plot the time delays for it separately.
wave_zero = oms_child_order_df_unpacked[
    oms_child_order_df_unpacked["wave_id"] == 0
]
# Get the timestamps of events for each child order
wave_zero_events = obccexqu.get_oms_child_order_timestamps(wave_zero)
wave_zero_events = wave_zero_events.sort_values(
    wave_zero_events.first_valid_index(), axis=1
)

In [ ]:
wave_zero_events.head(3)

In [ ]:
# Get the difference between event timestamps.
time_delays = obccexqu.get_time_delay_between_events(wave_zero_events)

In [ ]:
time_delays.boxplot(rot=45, ylabel="Time delay").set_title(
    "Time delay between events for wave 0"
)

### Plot the rest of the waves

In [ ]:
# Plot non-zero wave time to get the "average" time of the
# order submission, without computing portfolio and forecasts.
non_wave_zero_events = oms_child_order_df_unpacked[
    oms_child_order_df_unpacked["wave_id"] > 0
]
# Get the timestamps of events for each child order
non_wave_zero_events = obccexqu.get_oms_child_order_timestamps(
    non_wave_zero_events
)
non_wave_zero_events = non_wave_zero_events.sort_values(
    non_wave_zero_events.first_valid_index(), axis=1
)

In [ ]:
non_wave_zero_events.head(3)

In [ ]:
# Get the difference between event timestamps.
time_delays = obccexqu.get_time_delay_between_events(non_wave_zero_events)

In [ ]:
time_delays.boxplot(rot=45, ylabel="Time delay").set_title(
    "Time delay between events for waves >0"
)